In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


In [2]:
# Define directory names
train_data_dir = 'chest_xray/train/'
val_data_dir = 'chest_xray/val/'
test_data_dir = 'chest_xray/test/'

In [3]:
# Get all the data in the directory data/train (5216 images), and reshape them into (64, 64)
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=5216)

val_generator = ImageDataGenerator().flow_from_directory(
        val_data_dir, 
        target_size=(64, 64), batch_size=16)

test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, 
        target_size=(64, 64), batch_size=624)

# Create the datasets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)
test_images, test_labels = next(test_generator)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(val_images))
print(np.shape(val_labels))

(5216, 64, 64, 3)
(5216, 2)
(16, 64, 64, 3)
(16, 2)


In [5]:
train_img_unrow = train_images.reshape(5216, -1).T
val_img_unrow = val_images.reshape(16, -1).T
test_img_unrow = test_images.reshape(16, -1).T
np.shape(train_img_unrow)

(12288, 5216)

In [6]:
train_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [7]:
train_labels_final = train_labels.T[[1]]
val_labels_final = val_labels.T[[1]]
test_labels_final = test_labels.T[[1]]
np.shape(train_labels_final)

(1, 5216)

In [8]:
train_img_final = (train_img_unrow/255).astype('float32')
val_img_final = (val_img_unrow/255).astype('float32')
test_img_final = (test_img_unrow/255).astype('float32')
train_img_final.shape

(12288, 5216)

# Baseline - Logistic Regression

In [9]:
# Initialize empyty array with size N
def init_w(n):
    w = np.zeros((n, 1))
    return w

In [10]:
# Returns the cost from the current gradient
def propagation(w, b, x, y):
    l = x.shape[1]
    y_hat = 1/(1 + np.exp(- (np.dot(w.T, x) + b)))                                  
    cost = -(1/l) * np.sum(y * np.log(y_hat) + (1-y)* np.log(1 - y_hat))    
    dw = (1/l) * np.dot(x,(y_hat - y).T)
    db = (1/l) * np.sum(y_hat - y)
    return dw, db, cost

In [11]:
# Defines the learning rate and repeatedly calls the propagation function to "gradually" decrease to the lowest cost
def optimization(w, b, x, y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        dw, db, cost = propagation(w, b, x, y)    
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        # Record the costs and print them every 50 iterations
        if i % 50 == 0:
            costs.append(cost)
        if print_cost and i % 50 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return w, b, costs

In [12]:
# Makes predictions in our data, if y_hat > 0.5, then image is classified as 'PNEUMONIA'
def prediction(w, b, x):
    l = x.shape[1]
    y_prediction = np.zeros((1, l))
    w = w.reshape(x.shape[0], 1)
    y_hat = 1/(1 + np.exp(- (np.dot(w.T, x) + b))) 
    p = y_hat
    
    for i in range(y_hat.shape[1]):
        if (y_hat[0,i] > 0.5): 
            y_prediction[0, i] = 1
        else:
            y_prediction[0, i] = 0
    return y_prediction

In [13]:
# Combines the above functions to create a logistic regression model by entering train, test datasets, the number of 
# iterations and learning rate.
def model(x_train, y_train, x_test, y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    b = 0
    w = init_w(np.shape(x_train)[0]) 

    # Gradient descent (≈ 1 line of code)
    w, b, costs = optimization(w, b, x_train, y_train, num_iterations, learning_rate, print_cost)
    
    y_pred_test = prediction(w, b, x_test)
    y_pred_train = prediction(w, b, x_train)

    # Print train/test errors
    print('train accuracy: {} %'.format(100 - np.mean(np.abs(y_pred_train - y_train)) * 100))
    print('test accuracy: {} %'.format(100 - np.mean(np.abs(y_pred_test - y_test)) * 100))

    output = {'costs': costs, 
              'y_pred_test': y_pred_test,  
              'y_pred_train' : y_pred_train,  
              'w' : w, 
              'b' : b, 
              'learning_rate' : learning_rate, 
              'num_iterations': num_iterations}
    
    return output

In [14]:
# Baseline - Baseline. 500 iterations at 0.01 learning rate
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=500, learning_rate=0.01, print_cost=False)

train accuracy: 95.0536809815951 %
test accuracy: 87.5 %


In [15]:
# Possibly overfitting, a lower number of iterations - at 300 - underfits our model by a tad, but not much
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=300, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 1.524936
Cost after iteration 100: 0.447747
Cost after iteration 150: 0.316015
Cost after iteration 200: 0.188705
Cost after iteration 250: 0.163196
train accuracy: 94.1909509202454 %
test accuracy: 87.5 %


In [16]:
# Decrease the learning rate while also lowering the number of iterations made our model fit not as much, but our metric 
# scores decreased drastically
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=200, learning_rate=0.005, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 0.497045
Cost after iteration 100: 0.326489
Cost after iteration 150: 0.242337
train accuracy: 92.4463190184049 %
test accuracy: 81.25 %


In [17]:
# In order to regain out testing accuracy metric of 87.5%, we re-increased the learning rate to 0.01
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=200, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 1.524936
Cost after iteration 100: 0.447747
Cost after iteration 150: 0.316015
train accuracy: 92.48466257668711 %
test accuracy: 87.5 %


In [19]:
# Score the best model on the testing data instead of the validation data
output = model(train_img_final, train_labels_final, test_img_final, test_labels_final, 
               num_iterations=200, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 1.524936
Cost after iteration 100: 0.447747
Cost after iteration 150: 0.316015


ValueError: cannot reshape array of size 12288 into shape (479232,1)

# MLP

In [20]:
train_images_mlp = (train_images / 255).astype('float32')
val_images_mlp = (val_images / 255).astype('float32')
test_images_mlp = (test_images / 255).astype('float32')

In [21]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(64, 64, 3),
                        data_format = 'channels_last'))
model_1.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_1.add(layers.Flatten())        
model_1.add(layers.Dense(128))
model_1.add(layers.Activation('relu'))
#model_1.add(layers.Dropout(0.25))
model_1.add(layers.Dense(2))
model_1.add(layers.Activation('sigmoid'))

In [22]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4194432   
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0

In [23]:
model_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model_1.fit(train_images_mlp, train_labels, epochs=5, batch_size=50)

Epoch 1/5
105/105 [==============================] - 4s 43ms/step - loss: 0.4333 - accuracy: 0.8273
Epoch 2/5
105/105 [==============================] - 4s 41ms/step - loss: 0.1521 - accuracy: 0.9448
Epoch 3/5
105/105 [==============================] - 4s 41ms/step - loss: 0.1135 - accuracy: 0.9571
Epoch 4/5
105/105 [==============================] - 4s 41ms/step - loss: 0.0901 - accuracy: 0.9666
Epoch 5/5
105/105 [==============================] - 4s 41ms/step - loss: 0.0693 - accuracy: 0.9770


In [30]:
val_loss, val_acc = model_1.evaluate(val_images_mlp, val_labels)

1/1 [==============================] - 0s 997us/step - loss: 0.4118 - accuracy: 0.7500


In [31]:
model_2 = models.Sequential()
model_2.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(64, 64, 3),
                        data_format = 'channels_last'))
model_2.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_2.add(layers.Flatten())        
model_2.add(layers.Dense(512))
model_2.add(layers.Activation('relu'))
#model_1.add(layers.Dropout(0.25))
model_2.add(layers.Dense(2))
model_2.add(layers.Activation('sigmoid'))
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               16777728  
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_3 (Activation)    (None, 2)                

In [32]:
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_2.fit(train_images_mlp, train_labels, epochs=5, batch_size=50)

Epoch 1/5
105/105 [==============================] - 8s 75ms/step - loss: 0.4999 - accuracy: 0.8562
Epoch 2/5
105/105 [==============================] - 9s 83ms/step - loss: 0.1120 - accuracy: 0.9592
Epoch 3/5
105/105 [==============================] - 8s 80ms/step - loss: 0.0927 - accuracy: 0.9661
Epoch 4/5
105/105 [==============================] - 9s 83ms/step - loss: 0.0716 - accuracy: 0.9747
Epoch 5/5
105/105 [==============================] - 9s 85ms/step - loss: 0.0591 - accuracy: 0.9801


In [33]:
val_loss, val_acc = model_2.evaluate(val_images_mlp, val_labels)

1/1 [==============================] - 0s 997us/step - loss: 0.1978 - accuracy: 0.9375


In [34]:
model_3 = models.Sequential()
model_3.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(64, 64, 3),
                        data_format = 'channels_last'))
model_3.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_3.add(layers.Flatten())        
model_3.add(layers.Dense(4096))
model_3.add(layers.Activation('relu'))
#model_1.add(layers.Dropout(0.25))
model_3.add(layers.Dense(2))
model_3.add(layers.Activation('sigmoid'))
model_3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        416       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              134221824 
_________________________________________________________________
activation_4 (Activation)    (None, 4096)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 8194      
_________________________________________________________________
activation_5 (Activation)    (None, 2)                

In [35]:
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_3.fit(train_images_mlp, train_labels, epochs=5, batch_size=50)

Epoch 1/5
105/105 [==============================] - 53s 506ms/step - loss: 0.7669 - accuracy: 0.8838
Epoch 2/5
105/105 [==============================] - 50s 479ms/step - loss: 0.0900 - accuracy: 0.9682
Epoch 3/5
105/105 [==============================] - 49s 469ms/step - loss: 0.0702 - accuracy: 0.9757
Epoch 4/5
105/105 [==============================] - 50s 476ms/step - loss: 0.0650 - accuracy: 0.9749
Epoch 5/5
105/105 [==============================] - 50s 477ms/step - loss: 0.0367 - accuracy: 0.9885


In [36]:
val_loss, val_acc = model_2.evaluate(val_images_mlp, val_labels)

1/1 [==============================] - 0s 996us/step - loss: 0.1978 - accuracy: 0.9375


# More Pixels!

In [37]:
more_pixels = 224, 224

In [38]:
val_generator = ImageDataGenerator().flow_from_directory(
        val_data_dir, 
        target_size=(more_pixels), batch_size=16)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(more_pixels), batch_size=5216)

test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, 
        target_size=(more_pixels), batch_size=624)

test_images, test_labels = next(test_generator)
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)

Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [39]:
train_images_mlp = (train_images / 255).astype('float32')
val_images_mlp = (val_images / 255).astype('float32')
test_images_mlp = (test_images / 255).astype('float32')

In [40]:
model_4 = models.Sequential()
model_4.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(224, 224, 3),
                        data_format = 'channels_last'))
model_4.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_4.add(layers.Flatten())        
model_4.add(layers.Dense(128))
model_4.add(layers.Activation('relu'))
#model_1.add(layers.Dropout(0.25))
model_4.add(layers.Dense(2))
model_4.add(layers.Activation('sigmoid'))

In [41]:
model_4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 224, 224, 32)      416       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               51380352  
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                

In [42]:
model_4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['Recall'])
model_4.fit(train_images_mlp, train_labels, epochs=5, batch_size=50)

Epoch 1/5
105/105 [==============================] - 52s 494ms/step - loss: 1.1843 - recall: 0.8639
Epoch 2/5
105/105 [==============================] - 52s 495ms/step - loss: 0.0815 - recall: 0.9741
Epoch 3/5
105/105 [==============================] - 55s 528ms/step - loss: 0.0543 - recall: 0.9791
Epoch 4/5
105/105 [==============================] - 53s 507ms/step - loss: 0.0403 - recall: 0.9873
Epoch 5/5
105/105 [==============================] - 52s 493ms/step - loss: 0.0277 - recall: 0.9921


In [43]:
val_loss, val_acc = model_4.evaluate(val_images_mlp, val_labels)

1/1 [==============================] - 0s 997us/step - loss: 0.1108 - recall: 0.9375


In [44]:
test_loss, test_acc = model_4.evaluate(test_images_mlp, test_labels)

20/20 [==============================] - 2s 81ms/step - loss: 0.8874 - recall: 0.7885
